In [45]:
import ast
import pandas as pd
import json

In [46]:
raw = pd.read_json("data/coviddata.json")

data = raw['values'].tolist()
df = pd.DataFrame(data)

In [47]:
new_cols = df.iloc[:1].values.tolist()[0]
df.columns = new_cols
clean_df = df[2:]
clean_df.head(2)

,Approved,Deduped,Mod Status,Timestamp,What is the name of the hospital or clinic?,Final Address,Street address for dropoffs?,City,State?,"Drop off instructions, eg curbside procedure or mailing address ATTN: instructions:",...,Will they accept open boxes/bags?,Type of request:,Email Address,Type of organization?,CCN / Hospital ID (if applicable),When will you start running out of supplies?,Please describe most significant shortages:,State?,Lat,Lng
2,x,x,,3/19/2020 11:14:25,Swedish Ballard,"5300 Tallman Ave NW\nSeattle, WA 98107","5300 Tallman Ave NW\nSeattle, WA 98107",Seattle,WA,Put in donations bin at registration desk or a...,...,Yes,,,,,,,,47.6674625,-122.3795306
3,x,x,,3/19/2020 14:37:04,Zuckerberg San Francisco General Hospital,"1001 Potrero Ave\nSan Francisco, CA 94110","1001 Potrero Ave\nSan Francisco, CA 94110",San Francisco,CA,"For now, call ahead: call the switchboard (628...",...,No,,,,,,,,37.7557265,-122.4047381


In [48]:
all_ppe = [ppe.split(',') for ppe in clean_df['What are they accepting?'].values.tolist()]

In [49]:
flatten = lambda l: [item for sublist in l for item in sublist]

flat_ppe = set(flatten(all_ppe))

In [50]:
store = dict()
for ppe in flat_ppe:
    store[ppe] = []

for idx, row in clean_df.iterrows():
    for ppe in store.keys():
        if ppe in row['What are they accepting?']:
            store[ppe].append(True)
        else:
            store[ppe].append(False)
            
store_df = pd.DataFrame(store)

In [51]:
common_ppe = pd.DataFrame(store_df.sum()).sort_values(0, ascending=False).iloc[:20]
list(common_ppe.index)

['',
 'N95s',
 'Surgical masks',
 ' Surgical masks',
 'Face shields',
 'Gowns',
 ' Gown',
 ' Face shields',
 ' Gowns',
 ' sanitize',
 ' sanitizer',
 'Hand sanitizer',
 ' Hand sanitizer',
 ' Safety goggles',
 'Gloves',
 ' Gloves',
 ' Disinfecting wipes',
 ' Disposable booties',
 ' Thermometers',
 ' Surgical Masks']

In [52]:
most_common_items = ['N95s',
 'Surgical masks',
 'Face shields',
 'Gowns',
 'Hand sanitizer',
 ' Safety goggles',
 'Gloves',
 ' Disinfecting wipes',
 ' Disposable booties',
 ' Thermometers',
 ' Surgical Masks']

In [53]:
merged_df = df.join(store_df[most_common_items])

In [54]:
merged_df.head(2)

,Approved,Deduped,Mod Status,Timestamp,What is the name of the hospital or clinic?,Final Address,Street address for dropoffs?,City,State?,"Drop off instructions, eg curbside procedure or mailing address ATTN: instructions:",...,Surgical masks,Face shields,Gowns,Hand sanitizer,Safety goggles,Gloves,Disinfecting wipes,Disposable booties,Thermometers,Surgical Masks
0,Approved,Deduped,Mod Status,Timestamp,What is the name of the hospital or clinic?,Final Address,Street address for dropoffs?,City,State?,"Drop off instructions, eg curbside procedure o...",...,False,False,False,False,False,False,False,False,False,True
1,approved,deduped,residence,timestamp,name,address,orig_address,city,state,instructions,...,False,False,False,False,False,False,False,False,False,True


In [35]:
export_df = merged_df.to_csv('data/findthemasksjson_parsed_03272020.csv')